In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from queue import PriorityQueue

In [2]:
N = None # number of vertices
E = [] # contain edge
k = None # number of clusters
clusters = [] # contains vertiecs of each cluster
a = [] # contain weight matrix

In [3]:
def read_data(dist):
    global N, E, clusters, k, a
    N = None
    E = []
    a = []
    clusters = []
    k = None
    with open(dist, 'r', encoding='utf-8') as f:
        print(f.readline())
        print(f.readline())
        N = int(f.readline().split(' ')[2])
        k = int(f.readline().split(' ')[1])
        print(f.readline())
        # print(f.readline())

        for i in range(N):
            tmp = f.readline().split()
            # print(tmp)
            for j in range(i+1, N):
                E.append([i, j, int(tmp[j])])
            
            a.append([])
            for j in range(N):
                a[i].append(int(tmp[j]))

        print(f.readline())
        # print(f.readline())
        for i in range(k):
            tmp = f.readline().split()
            tmp2 = []
            for j in range(1, len(tmp)-1):
                tmp2.append(int(tmp[j])-1)
            clusters.append(tmp2)
        print(clusters)
# read_data(r'Data\Type_1_Small\5berlin52.txt') 

In [4]:
class Chromosome:
    # n la so cum
    def __init__(self, n, arr = None):
        if arr is None:
            arr = np.arange(n, dtype = int)
            np.random.shuffle(arr)
            arr = arr.tolist()
        self.arr = arr
        self.fitness = -1
    
    def __call__(self):
        return self.arr
    
# test = Chromosome(k)
# print(test.arr)

In [5]:
# calculate cost of a tree of a cluster_id with cluster_id's req vertices and some non_req vertices
def cal_inner_cluster(cx, group):
    # group is id of that cluster_id
    global a
    global N
    res = 0
    Vt = []
    Et = []
    d = [1e18]*N
    p = [-1]*N
    s = cx.index(group)
    d[s] = 0
    qu = PriorityQueue()
    Vt.append(s)
    qu.put((d[s], s))

    while not qu.empty():
        du, u = qu.get()
        
        if du>d[u]:
            continue

        if cx[u] == group:
            v = u
            while v not in Vt:
                Vt.append(v)
                Et.append([p[v], v])
                d[v] = 0
                qu.put((d[v], v))
                v = p[v]
                
        for v in range(len(a[u])):
            if cx[v]==-1 or cx[v]==group:
                if d[u] + a[u][v] < d[v]:
                    d[v] = d[u] + a[u][v]
                    p[v] = u
                    qu.put((d[v], v))
    # mask vertices in group                
    for i in Vt:
        cx[i] = group
    for u, v in Et:
        res += a[u][v]
    
    return Vt, Et, res
                

In [6]:
# calculate cost between multiple clusters
def cal_inter_cluster(vt):
    global N
    res = 0
    Vt = []
    Et = []
    d = [1e18]*2*N
    p = [-1]*2*N
    s = 51
    d[s] = 0
    qu = PriorityQueue()
    Vt.append(s)
    qu.put((d[s], s))

    while not qu.empty():
        du, u = qu.get()
        
        if du>d[u]:
            continue

        if u >= N:
            v = u
            while v not in Vt:
                Vt.append(v)
                Et.append([p[v], v])
                res += d[v] - d[p[v]]
                d[v] = 0
                qu.put((d[v], v))
                v = p[v]
                    
        for v, w in vt[u]:
            if d[u] + w < d[v]:
                d[v] = d[u] + w
                p[v] = u
                qu.put((d[v], v))
 
    
    return Vt, Et, res
                

In [7]:
# calculate total cost of a chromosome
def cal_cost(chr: Chromosome):
    if chr.fitness!= -1:
        return chr.fitness
    
    global E, N
    cost = 0 # cost of chromosome

    cx = [-1]*N # note the vertex's cluster

    for i in range(len(clusters)):
        for j in clusters[i]:
            cx[j] = i

    # print(cx)
    # prim + dijsktra on each cluster
    for i in chr.arr: 
        _, _, tmp_cost = cal_inner_cluster(cx, i)
        cost += tmp_cost

    # contract thành một danh sách cạnh
    dict = {}
    for edge in E:
        u = edge[0]
        v = edge[1]
        w = edge[2]
        if cx[u] != -1:
            u = cx[u] + N
        if cx[v] != -1:
            v = cx[v] + N
        if u==v:
            continue

        # tạo edge và đỉnh kề cho từng đỉnh
        if u > v:
            u, v = v, u
        
        if (u, v) not in dict:
            dict[(u, v)] = w
        else:
            dict[(u, v)] = min(w, dict[(u, v)])

    # tạo danh sách cạnh kề
    vt = [[] for i in range(2*N)] 
    for key, item in dict.items():
        u = key[0]
        v = key[1]
        vt[u].append((v, item))
        vt[v].append((u, item))
        
    _ , _, inter_cost = cal_inter_cluster(vt)
    cost += inter_cost
    chr.fitness = cost
    return cost

# cal_cost(test)

In [8]:
def generate_chromosome(num):
    population = []
    global k
    for i in range(num):
        population.append(Chromosome(k))
    
    for i in range(len(population)):
        cal_cost(population[i])
    return population

In [9]:
def multation(chromo):
    global k
    father = chromo()
    point1, point2 = np.random.choice(np.arange(k), size=2, replace=False)
    tmp = father[point1]
    father[point1] = father[point2]
    father[point2] = tmp
    new_chromo = Chromosome(N)
    new_chromo.arr = father
    return new_chromo

# print(test())
# print(multation(test).arr)

In [10]:
def doublePointCrossover(chromo1, chromo2):
    par1_city = chromo1()
    par2_city = chromo2()

    n = len(par1_city)
    
    i1, i2 = 0, 0
    while i1==i2:
        points = np.random.choice(range(len(par1_city) + 1), size = 2, replace=False) # lay diem cat
        i1 = points[0]
        i2 = points[1]
    
    child1_city = [None] * n
    child2_city = [None] * n
    if i1<i2:
        child1_city[i1: i2] = par1_city[i1: i2]
        child2_city[i1: i2] = par2_city[i1: i2]
    else:
        child1_city[i1: ], child1_city[: i2] = par1_city[i1: ], par1_city[ : i2]
        child2_city[i1: ], child2_city[: i2] = par2_city[i1: ], par2_city[ : i2]

    remaining_par1 = [x for x in par1_city if x not in child2_city] # những thằng có trong par1 k có trong child2
    remaining_par2 = [x for x in par2_city if x not in child1_city]

    # print(child2_city)
    # print(remaining_par1)

    for i in range(n):
        if child1_city[i] is None:
            child1_city[i] = remaining_par2.pop(0)
            

    for i in range(n):
        if child2_city[i] is None:
            child2_city[i] = remaining_par1.pop(0)

    new_chromo1 = Chromosome(n, arr = child1_city)
    new_chromo2 = Chromosome(n, arr =  child2_city)
    return new_chromo1, new_chromo2

In [11]:
def choose_best(parent_pop, children_pop):
    n = len(parent_pop)
    mixed_pop = parent_pop + children_pop
    for i in range(len(children_pop)):
        cal_cost(children_pop[i])
    for i in range(len(parent_pop)):   
        cal_cost(parent_pop[i])
        
    mixed_pop = sorted(mixed_pop, key = lambda x: x.fitness)
    return mixed_pop[:n]

In [12]:
def cross_over_population(num_pop, parent_pop, multation_rate = 0.2, rate = 0.1):
    for i in range(len(parent_pop)):
        cal_cost(parent_pop[i])
    child_pop = []
    num_to_rand = int(num_pop*rate) # số lượng lấy sample
    for i in range(num_pop):
        tmp_par = np.random.choice(parent_pop, size=num_to_rand, replace=False)
        tmp_par = sorted(tmp_par, key = lambda x: x.fitness)
        parent1, parent2 = tmp_par[0: 2]
        child1, child2 = doublePointCrossover(parent1, parent2)
        random_value = np.random.rand()
        if random_value <= multation_rate:
            child1 = multation(child1)
            child2 = multation(child2)
        
        child_pop.append(child1)
        child_pop.append(child2)
    return child_pop

In [13]:
# calculate cost for 1 population after some generation with 1 certain seed
def main(num_pop = 100, num_gen = 500, mutation_rate = 0.2, seed = 42):
    np.random.seed(seed=seed)
    global N
    population = generate_chromosome(num_pop)
    for gen in range(num_gen):
        child_pop = cross_over_population(num_pop, population, multation_rate=mutation_rate, rate=0.1)
        population = choose_best(parent_pop=population, children_pop=child_pop)
        # print(f'gen {gen}: {population[0].fitness}, {population[0].arr[:10]},\n {child_pop[0].fitness}, {child_pop[0].arr[:10]}')
    # print(f'seed: {seed}, result = {population[0].fitness}, {population[0].arr[:10]}')
    return population[0].fitness


In [14]:
# calculate cost for multi_seed (eg: from 1 to 30)
def cost_avg(seeds = 30):
    start_time = time.time()
    avg_fitness = 0
    best_fitness = 1e19
    for i in range(seeds):
        print(f'Seed {i} is running...')
        tmp_fitness = main(num_pop=100, num_gen=50, mutation_rate=0.2, seed=i)
        best_fitness = min(best_fitness, tmp_fitness)
        avg_fitness += tmp_fitness
    avg_fitness /= seeds
    end_time = time.time()
    execution_time = round(end_time - start_time, 3)
    print(f'Over 30 seeds, avg_fitness: {avg_fitness}, best_fitness: {best_fitness}, time(s): {execution_time}')
    return avg_fitness
file_input = "/kaggle/input/steiner/datasteiner/10eil51.txt"
read_data(file_input)
cost_avg()

Name : 10eil51

TYPE: NON_EUC_CLUSTERED_TREE

EDGE_WEIGHT_SECTION:

CLUSTER_SECTION:

[[5], [12], [0, 25, 30], [3, 24], [4], [10], [14, 36, 43], [1, 49], [2, 19], [9]]
Seed 0 is running...
Seed 1 is running...
Seed 2 is running...
Seed 3 is running...
Seed 4 is running...
Seed 5 is running...
Seed 6 is running...
Seed 7 is running...
Seed 8 is running...
Seed 9 is running...
Seed 10 is running...
Seed 11 is running...
Seed 12 is running...
Seed 13 is running...
Seed 14 is running...
Seed 15 is running...
Seed 16 is running...
Seed 17 is running...
Seed 18 is running...
Seed 19 is running...
Seed 20 is running...
Seed 21 is running...
Seed 22 is running...
Seed 23 is running...
Seed 24 is running...
Seed 25 is running...
Seed 26 is running...
Seed 27 is running...
Seed 28 is running...
Seed 29 is running...
Over 30 seeds, avg_fitness: 887.0, best_fitness: 887, time(s): 3542.381


887.0